In [39]:
from bs4 import BeautifulSoup
# import time
import requests
# import json
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service
import pandas as pd
import re
# from concurrent.futures import ThreadPoolExecutor

In [40]:
#define variables

url = "https://letterboxd.com/films/popular/decade/2000s/page/"
start_page = 1
page_count = 7
soup = 0
link = 0

In [41]:
#To parse the given page
def parse_page(url = url):
    options = Options()
    options.add_argument("--headless")
    service = Service(executable_path= r"C:\Users\abhyu\Downloads\geckodriver-v0.32.1-win64\geckodriver.exe")
    driver = webdriver.Firefox(options=options, service=service)

    driver.get(url)

    soup = BeautifulSoup(driver.page_source, "html.parser")
    driver.quit()

    # print(soup.prettify)
    
    return soup

In [42]:
def parse_without_selenium(url = url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")

    return soup

In [43]:
#data for film in individual page

def more_film_data(link=link):
    film_soup = parse_without_selenium(link)
    
    actor_data = film_soup.find_all('a', {"class" : "text-slug tooltip"}, limit= 3)
    actors = []
    for actor in actor_data:
        actors.append(actor.text)

    director = film_soup.find('span', {"class" : "prettify"}).text

    # country_or_countries = film_soup.find()
    if film_soup.find('h3', text=re.compile(r'Country|Countries')):
        country_tag = film_soup.find('h3', text=re.compile(r'Country|Countries')).find_next('div')
        country = [a.text for a in country_tag.find_all('a')]
    else:
        country = None

    language = film_soup.find('h3', text=re.compile(r'Original Language|Language')).find_next('div').text.strip()

    rating = film_soup.find('meta', {"name" : "twitter:data2"})["content"]
    rating = float(rating.replace(' out of 5',''))

    stat_soup = parse_without_selenium(link + '/members')
    letterboxd_views = stat_soup.find('a', class_="tooltip")['title']
    letterboxd_views = int(''.join(filter(str.isdigit, letterboxd_views)))

    #Genre
    genres_data = film_soup.find('div', class_="text-sluglist capitalize")
    genres = []
    for genre in genres_data.find_all('a'):
        genres.append(genre.text.strip())

    #Number of films in the franchise
    if film_soup.find('section', {"id" : "related"}):
        franchise = "yes"
    else:
        franchise = "no"

    #LINKS FOR OTHER SITES
    imdb_link = film_soup.find('a', {"data-track-action" : "IMDb"})['href']
    tmdb_link = film_soup.find('a', {"data-track-action" : "TMDb"})['href']

    more_film_list = [rating, letterboxd_views, director, country, language, actors, genres, franchise, imdb_link, tmdb_link]
    # more_film_list.append(actors)

    
    return more_film_list

In [44]:
#Create a dataframe of all films in a given page
def collect_data(soup = soup):
    # ratings = soup.find_all('li', {"class" : "listitem poster-container"}) 
    list_of_movies = soup.find_all('div', {"data-component-class" : "globals.comps.FilmPosterComponent"}) 

    df_page = pd.DataFrame()
    

    #One movie at a time
    for index, attributes in enumerate(list_of_movies):
        # print(attributes.get('data-film-name')+', '+ attributes.get('data-film-release-year'))
        link = 'https://letterboxd.com' + attributes.get('data-film-link')
        row = [attributes.text, attributes.get('data-film-name'), attributes.get('data-film-release-year'), link]
        print(attributes.get('data-film-name'))

        more_film_list = more_film_data(link)
        row.extend(more_film_list)
        
        # print(row)
        df_row = pd.DataFrame(row)

        df_page = pd.concat([df_page, df_row], axis=1)

        #Testing
        # if index == 3:
        #     break
    
    
    return df_page
        
    

In [45]:
# loop over the pages
def page_loop(url=url, page_count=page_count, start_page = start_page):

    df = pd.DataFrame()
    
    for count in range(start_page, page_count):
        page_url = url + str(count)
        print(page_url)
        soup = parse_page(page_url)
        df_page = collect_data(soup)
        df = pd.concat([df, df_page.transpose()])
        export_address = fr'C:\Users\abhyu\Desktop\Dissertation\fetchMovies_Letterboxd\film_data\film_data{count}.csv'
        # print(export_address)
        df.to_csv(export_address, index = False)
        # print(count)
    return df
        

In [46]:
def get_data(url=url, page_count=page_count):
    df = page_loop(url, page_count)
    # df.columns = ["Name and Year", "Name", "Year", "Link", "Letterboxd Rating", "Letterboxd Views", "Director", "Country", "Language", "Actors" ]
    return df

In [47]:
df = get_data(url, page_count+1)
df

https://letterboxd.com/films/popular/decade/2000s/page/1
The Dark Knight
American Psycho
Spirited Away
Eternal Sunshine of the Spotless Mind
Inglourious Basterds
Kill Bill: Vol. 1
Ratatouille
The Lord of the Rings: The Fellowship of the Ring
Donnie Darko
Fantastic Mr. Fox
Avatar
Up
WALL·E
No Country for Old Men
Harry Potter and the Prisoner of Azkaban
Iron Man
Little Miss Sunshine
Harry Potter and the Philosopher's Stone
Spider-Man
Memento
Zodiac
The Incredibles
The Lord of the Rings: The Return of the King
Howl's Moving Castle
(500) Days of Summer
Finding Nemo
The Lord of the Rings: The Two Towers
Catch Me If You Can
Spider-Man 2
Monsters, Inc.
Shrek
Lost in Translation
Requiem for a Dream
Batman Begins
Mean Girls
There Will Be Blood
Harry Potter and the Goblet of Fire
Harry Potter and the Chamber of Secrets
Superbad
Coraline
The Prestige
Amélie
The Departed
Star Wars: Episode III - Revenge of the Sith
Mulholland Drive
Harry Potter and the Order of the Phoenix
Oldboy
Kill Bill: Vol. 2

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,The Dark Knight (2008),The Dark Knight,2008,https://letterboxd.com/film/the-dark-knight/,4.45,2258674,Christopher Nolan,"[UK, USA]",English,"[Christian Bale, Heath Ledger, Michael Caine]","[action, thriller, drama, crime]",yes,http://www.imdb.com/title/tt0468569/maindetails,https://www.themoviedb.org/movie/155/
0,American Psycho (2000),American Psycho,2000,https://letterboxd.com/film/american-psycho/,3.93,1649310,Mary Harron,"[Canada, USA]",English,"[Christian Bale, Willem Dafoe, Jared Leto]","[drama, thriller, crime]",yes,http://www.imdb.com/title/tt0144084/maindetails,https://www.themoviedb.org/movie/1359/
0,Spirited Away (2001),Spirited Away,2001,https://letterboxd.com/film/spirited-away/,4.48,1684343,Hayao Miyazaki,[Japan],Japanese,"[Rumi Hiiragi, Miyu Irino, Mari Natsuki]","[fantasy, animation, family]",no,http://www.imdb.com/title/tt0245429/maindetails,https://www.themoviedb.org/movie/129/
0,Eternal Sunshine of the Spotless Mind (2004),Eternal Sunshine of the Spotless Mind,2004,https://letterboxd.com/film/eternal-sunshine-o...,4.23,1507318,Michel Gondry,[USA],English,"[Jim Carrey, Kate Winslet, Kirsten Dunst]","[drama, romance, science-fiction]",no,http://www.imdb.com/title/tt0338013/maindetails,https://www.themoviedb.org/movie/38/
0,Inglourious Basterds (2009),Inglourious Basterds,2009,https://letterboxd.com/film/inglourious-basterds/,4.29,1771967,Quentin Tarantino,"[France, Germany, USA]",English,"[Brad Pitt, Mélanie Laurent, Christoph Waltz]","[thriller, drama, war]",no,http://www.imdb.com/title/tt0361748/maindetails,https://www.themoviedb.org/movie/16869/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,The Girl Next Door (2004),The Girl Next Door,2004,https://letterboxd.com/film/the-girl-next-door...,2.94,137817,Luke Greenfield,[USA],English,"[Emile Hirsch, Elisha Cuthbert, Timothy Olyphant]","[romance, comedy]",no,http://www.imdb.com/title/tt0265208/maindetails,https://www.themoviedb.org/movie/10591/
0,Talk to Her (2002),Talk to Her,2002,https://letterboxd.com/film/talk-to-her/,3.84,84630,Pedro Almodóvar,"[Spain, USA]",Spanish,"[Leonor Watling, Rosario Flores, Javier Cámara]","[drama, romance]",no,http://www.imdb.com/title/tt0287467/maindetails,https://www.themoviedb.org/movie/64/
0,Star Wars: The Clone Wars (2008),Star Wars: The Clone Wars,2008,https://letterboxd.com/film/star-wars-the-clon...,2.74,138452,Dave Filoni,[USA],English,"[Matt Lanter, Ashley Eckstein, James Arnold Ta...","[animation, action, adventure, science-fiction]",no,http://www.imdb.com/title/tt1185834/maindetails,https://www.themoviedb.org/movie/12180/
0,3 Idiots (2009),3 Idiots,2009,https://letterboxd.com/film/3-idiots/,3.96,118195,Rajkumar Hirani,[India],Hindi,"[Aamir Khan, R. Madhavan, Sharman Joshi]","[drama, comedy]",no,http://www.imdb.com/title/tt1187043/maindetails,https://www.themoviedb.org/movie/20453/


In [48]:
df.to_csv(fr'C:\Users\abhyu\Desktop\Dissertation\fetchMovies_Letterboxd\film_data\df_final.csv', index = False)

In [49]:
# url = "https://letterboxd.com/film/joker-2019//"

# # film_soup = parse_page(url)

# r = requests.get(url)
# film_soup = BeautifulSoup(r.text, "html.parser")


In [50]:
# list = more_film_data(url)
# print(list)
